In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle

import nltk
import string
from pymongo import MongoClient

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics

from collections import OrderedDict

import random

In [3]:
client = MongoClient()
db = client.legislation
bills = db.bills

In [4]:
# using the sentences in a single document to make clusters with custom tokenizer
with open('all_bills.pickle', 'rb') as f:
    bills_dict = pickle.load(f)


In [16]:
downsized = random.sample(bills_dict.items(), 5000)
texts = [i[1] for i in downsized]

In [17]:

vectorizer = CountVectorizer(min_df=1, stop_words='english', ngram_range=(1,3))

# selection = list(cursor)
# texts = [i['text_versions'].itervalues().next() for i in selection]
docs = vectorizer.fit_transform(texts)

# for i, selection in enumerate(cursor):
#     text = selection['text_versions'].itervalues().next()
#     vectorizer.fit([text])

In [18]:
docs.shape

(5000, 4351533)

In [ ]:
# with open('count_vectorized_all_docs', 'wb') as f:
#     pickle.dump(docs, f)

In [10]:
# # choose best k plot
# inertia_list = []
# for k in range(1, 100, 5):
#     print k
#     km = MiniBatchKMeans(init='k-means++', n_init=3,
#                          batch_size=100, n_clusters=k)
#     model = km.fit(X)
#     inertia_list.append((k, model.inertia_))
# plt.plot(zip(*inertia_list)[0],zip(*inertia_list)[1])

In [19]:
X = docs
k = 45
km = MiniBatchKMeans(init='k-means++', n_init=1, verbose=1, n_clusters=k)
km.fit(X)

Init 1/1 with method: k-means++
Inertia for init 1/1: 19549712.647421
Minibatch iteration 1/5000: mean batch inertia: 51642.541948, ewa inertia: 51642.541948 
Minibatch iteration 2/5000: mean batch inertia: 232132.800887, ewa inertia: 58860.708672 
Minibatch iteration 3/5000: mean batch inertia: 68530.669558, ewa inertia: 59247.429763 
Minibatch iteration 4/5000: mean batch inertia: 33940.074656, ewa inertia: 58235.337977 
Minibatch iteration 5/5000: mean batch inertia: 17550.441192, ewa inertia: 56608.267520 
Minibatch iteration 6/5000: mean batch inertia: 26542.866148, ewa inertia: 55405.891940 
Minibatch iteration 7/5000: mean batch inertia: 18342.072582, ewa inertia: 53923.635617 
Minibatch iteration 8/5000: mean batch inertia: 8870.296661, ewa inertia: 52121.862414 
Minibatch iteration 9/5000: mean batch inertia: 20360.764218, ewa inertia: 50851.672524 
[MiniBatchKMeans] Reassigning 40 cluster centers.
Minibatch iteration 10/5000: mean batch inertia: 10709.707901, ewa inertia: 492

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=45,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=1)

In [20]:
for cluster in range(k):
    cluster_documents = np.where(km.labels_  == cluster)[0]
    print("CLUSTER NUMBER {0}".format(cluster))
    closest = (-1, float('inf'))
    for index in cluster_documents:
        distance = pairwise_distances(X[index], km.cluster_centers_[cluster], metric='euclidean')
        if distance[0][0] < closest[1]:
            closest = (index, distance[0][0])
    print closest
    print texts[closest[0]]
    print("{0} similar documents were gathered".format(len(cluster_documents)))
    print

CLUSTER NUMBER 0
(92, 0.0)

[Congressional Bills 111th Congress]
[From the U.S. Government Printing Office]
[H.R. 6417 Introduced in House (IH)]

111th CONGRESS
  2d Session
                                H. R. 6417

  To prohibit Federal funding of certain public radio programming, to 
 provide for the transfer of certain public radio funds to reduce the 
                  public debt, and for other purposes.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           November 17, 2010

 Mr. Lamborn introduced the following bill; which was referred to the 
                    Committee on Energy and Commerce

_______________________________________________________________________

                                 A BILL


 
  To prohibit Federal funding of certain public radio programming, to 
 provide for the transfer of certain public radio funds to reduce the 
                  public d

In [ ]:
try stemming